In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql import DataFrameNaFunctions as DFna
from pyspark.sql.functions import udf, col, when
import matplotlib.pyplot as plt
import pyspark as ps
import os, sys, requests, json
from pyspark.sql.functions import col,size,regexp_replace,lit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import Row
import numpy as np
import math
from pyspark.sql.functions import regexp_replace
from pyspark.sql import *



spark = ( SparkSession.builder.appName("ModelTraining") .config("spark.executor.memory","16g") .getOrCreate() ) 
            
sc = spark.sparkContext 

In [ ]:
import numpy as np
import pandas as pd

from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, Tokenizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import IDF

from pyspark.ml import Pipeline, PipelineModel


from pyspark.sql.functions import *

from pyspark.sql.types import *

import folium
import html

In [ ]:
pro=spark.read.csv('../input/data-science-for-good-careervillage/professionals.csv', header=True,quote='"',sep=",",multiLine=True) 
email=spark.read.csv('../input/data-science-for-good-careervillage/emails.csv', header=True,quote='"',sep=",",multiLine=True) 
ques=spark.read.csv('../input/data-science-for-good-careervillage/questions.csv', header=True,quote='"',sep=",",multiLine=True) 
match=spark.read.csv('../input/data-science-for-good-careervillage/matches.csv', header=True,quote='"',sep=",",multiLine=True) 
ans=spark.read.csv('../input/data-science-for-good-careervillage/answers.csv', header=True,quote='"',sep=",",multiLine=True) 
ans_score=spark.read.csv('../input/data-science-for-good-careervillage/answer_scores.csv', header=True,quote='"',sep=",",multiLine=True) 


In [ ]:
#match_ques=match.join(ques,match.matches_question_id==ques.questions_id)
#match_ques=match_ques.join(ans,match_ques.questions_id==ans.answers_question_id)
#match_ques=match_ques.join(email,match_ques.matches_email_id==email.emails_id)
#match_ques=match_ques.join(pro,match_ques.emails_recipient_id==pro.professionals_id)


In [ ]:
#ques_ans=match_ques.select('questions_id','questions_body','answers_id','answers_body','professionals_id')

In [ ]:
#ques_ans=ques_ans.join(ans_score,ques_ans.answers_id==ans_score.id).select('questions_id','questions_body','professionals_id','score')

In [ ]:
#ques_ans=ques_ans.select('questions_id','questions_body','professionals_id','score')

In [ ]:
#Data cleaning question
def datacleaning_que(test):
    user_regex=r"(@\w{1,15})"
    clean_test=test.withColumn( 'user_mentioned',f.array_remove(
        f.array(
            f.regexp_extract(f.col("Full_questions"),user_regex,1),
            f.regexp_extract(
                f.col("Full_questions"), "".join([f"{user_regex}.*?" for i in range(0,2)]),2),
                f.regexp_extract(
                f.col("Full_questions"), "".join([f"{user_regex}.*?" for i in range(0,3)]),3),
                f.regexp_extract(
                f.col("Full_questions"), "".join([f"{user_regex}.*?" for i in range(0,4)]),4),
                f.regexp_extract(
                f.col("Full_questions"), "".join([f"{user_regex}.*?" for i in range(0,5)]),5),
                f.regexp_extract(
                f.col("Full_questions"), "".join([f"{user_regex}.*?" for i in range(0,6)]),6),), "",).alias('user_mentioned'))
    clean_test=clean_test.withColumn( 'original_text',f.col("Full_questions")).withColumn('Full_questions',f.regexp_replace(f.col("Full_questions"), user_regex,"").alias("Full_questions"))
    
    url_regex=r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%-~|$?!:,.]*)|(www.)+([-\w+&@#/%-~|$?!:,.]*)"
    clean_test=clean_test.withColumn( 'Full_questions',f.regexp_replace(f.col("Full_questions"), url_regex,""))
    #clean html
    clean_test = clean_test.withColumn('Full_questions', f.regexp_replace(f.col("Full_questions"), r'<.*?>', ''))
    
    email_regex=r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"
    clean_test=clean_test.withColumn('Full_questions',f.regexp_replace(f.col("Full_questions"), email_regex,""))

    test_cleaned=(clean_test.withColumn( 'Full_questions',f.regexp_replace(f.col("Full_questions"),"[^a-zA-Z]"," ")).withColumn( 'Full_questions',f.regexp_replace(f.col("Full_questions")," +"," ")).withColumn( 'Full_questions',f.trim(f.col("Full_questions"))).filter("Full_questions !=''"))
    test_data=test_cleaned.select('questions_id','Full_questions','questions_author_id').coalesce(3).cache()
    return test_data

In [ ]:
#a=f.concat(f.col("questions_title"),f.lit(' '),f.col("questions_body"))
#ques=ques.withColumn("Full_questions",a).alias("Full")

In [ ]:
ques=ques.withColumnRenamed("questions_body","Full_questions")
ques.show()

In [ ]:
ques = datacleaning_que(ques)
ques.show(20)

In [ ]:
ques_ans_final = ques.dropDuplicates().select('questions_id','Full_questions')
ques_ans_final=ques_ans_final.na.drop(subset=["Full_questions"])

In [ ]:
from pyspark.sql.functions import lit,row_number,col
from pyspark.sql.window import Window

w = Window().partitionBy(lit('a')).orderBy(lit('a'))

ques_ans_final = ques_ans_final.withColumn("row_num", row_number().over(w))

In [ ]:
# Build the pipeline 
tokenizer = RegexTokenizer(gaps = False, pattern = '\w+',inputCol = 'Full_questions', outputCol = 'token')
stopWordsRemover = StopWordsRemover(inputCol = 'token', outputCol = 'nostopwrd')
countVectorizer = CountVectorizer(inputCol="nostopwrd", outputCol="rawFeature")
iDF = IDF(inputCol="rawFeature", outputCol="idf_vec")
word2Vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'nostopwrd', outputCol = 'word_vec', seed=123)
vectorAssembler = VectorAssembler(inputCols=['idf_vec', 'word_vec'], outputCol='comb_vec')
pipeline = Pipeline(stages=[tokenizer, stopWordsRemover, countVectorizer, iDF, word2Vec, vectorAssembler])

# fit the model
pipeline_mdl = pipeline.fit(ques_ans_final)

In [ ]:
ques_ans_final.show(5)

In [ ]:
# transform the question data

ques_pipeline_df = pipeline_mdl.transform(ques_ans_final)

In [ ]:
def CosineSim(vec1, vec2): 
    return np.dot(vec1, vec2) / np.sqrt(np.dot(vec1, vec1)) / np.sqrt(np.dot(vec2, vec2))

In [ ]:
all_ques_vecs = ques_pipeline_df.select('questions_id', 'word_vec').rdd.map(lambda x: (x[0], x[1])).collect()

In [ ]:
def getQuestionDetails(in_ques):
    
    a = in_ques.alias("a")
    b = ques.alias("b")
    
    return a.join(b, col("a.question_id") == col("b.questions_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.Full_questions')])
    

In [ ]:
def getKeyWordsRecoms(key_words, sim_bus_limit):
    
    #print('\nQuestion Relevent "' + key_words + '"')
    
    input_words_df = sc.parallelize([(0, key_words)]).toDF(['question_id', 'Full_questions'])
    
    # transform the the key words to vectors
    input_words_df = pipeline_mdl.transform(input_words_df)
    
    # choose word2vec vectors
    input_key_words_vec = input_words_df.select('word_vec').collect()[0][0]
    
    # get similarity
    sim_bus_byword_rdd = sc.parallelize((i[0], float(CosineSim(input_key_words_vec, i[1]))) for i in all_ques_vecs)

    sim_bus_byword_df = spark.createDataFrame(sim_bus_byword_rdd) \
         .withColumnRenamed('_1', 'question_id') \
         .withColumnRenamed('_2', 'score') \
         .orderBy("score", ascending = False)

    sim_bus_byword_df=sim_bus_byword_df.na.drop(subset=["score"])
     # return top 10 similar 
    a = sim_bus_byword_df.limit(sim_bus_limit)
    return getQuestionDetails(a).sort('score', ascending=False)

In [ ]:
ans_score=ans_score.withColumnRenamed("score","vote")

In [ ]:
def acc_1ques(id_ques):
    
    key_words = str(np.array(ques.filter(ques.questions_id==id_ques).select('Full_questions').collect())[0][0])
    keywords_recom_df = getKeyWordsRecoms(key_words, 20)
    
    list_truth=np.array(ans.filter(ans.answers_question_id==id_ques).select('answers_author_id').collect())
    
    rs1 = keywords_recom_df.join(ans,keywords_recom_df.question_id==ans.answers_question_id).select("question_id","answers_id","answers_author_id","score")
        # top vote 20
    rs2 = rs1.join(ans_score,rs1.answers_id==ans_score.id).select("answers_author_id","question_id","vote","score")
    rs2 = rs2.withColumn("ts",rs2.vote*rs2.score)
    rs2 = rs2.sort('ts', ascending=False).limit(20)
        # new top 20 : 0.9807
    #rs2 = rs1.sort('score', ascending=False).limit(20) 
        # new top 20 vote >0: 0.4219
    #rs2 = rs1.join(ans_score,rs1.answers_id==ans_score.id).select("answers_author_id","question_id","score","vote")
    #rs2=rs2.filter(rs2.vote>0)
    #rs2 = rs2.sort('score', ascending=False).limit(20)
        # Remove 1.0 score
    #rs1 = rs1.filter(rs1.question_id!=id_ques) 
    #rs2 = rs1.sort('score', ascending=False).limit(20) 
    rs3 = rs2.join(pro,rs2.answers_author_id==pro.professionals_id).select("professionals_id","professionals_industry","professionals_headline")
    list_recom=np.array(rs3.select('professionals_id').dropDuplicates().collect())
    count=0
    sum_count=ans.filter(ans.answers_question_id==id_ques).count()
    for i in list_recom:
        for j in list_truth:
            if i==j: count=count+1
    count=float(count/sum_count)
    print(count)
    return count

In [ ]:
def score(test):
    sum=0
    for t in test:
        sum=sum+acc_1ques(t)
    score = float(sum/len(test))
    return score

In [ ]:
test=['01352c4d67fe435ca59e745ff2520d2a',

'03eee1ca07174470b160717027ab46d6',

'04a979f4e7fd49b9a07b6fae7a5727ee',

'062f49f153de4b8793e4e669ec5b5331',

'083965c88d894a9f9e4e71e521641338',

'09e3bdc69a6149aa8656bbc18162ac37',

'0d7fab391dc145a384da4af0a078b77f',

'0db6ed5d24df42f18d19958ccb32cd6e',

'1a039cb9f3064f76b386f84f303edc43',

'1a444e5e5824446eaf37f31effd72ce0']

In [ ]:
a = score(test)
a